In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .appName('init_env') \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .getOrCreate()

#Ajuste de log WARN log para ERROR
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/13 17:30:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Criar Schemas
schema_list = ['bronze','silver','gold']

for schema in schema_list:
    spark.sql(f"""
        CREATE DATABASE IF NOT EXISTS iceberg.{schema} COMMENT '' LOCATION 's3a://datalake/iceberg/{schema}/'
    """)

In [3]:
### Listar catalogs
spark.sql("SHOW databases").show()

+---------+
|namespace|
+---------+
|   bronze|
|  default|
|     gold|
|   silver|
+---------+



In [5]:
### Criar tabela teste
query = f"""CREATE TABLE IF NOT EXISTS iceberg.bronze.nyc_taxis
(
  vendor_id bigint,
  trip_id bigint,
  trip_distance float,
  fare_amount double,
  store_and_fwd_flag string
)
PARTITIONED BY (vendor_id);"""
spark.sql(query)

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


DataFrame[]

In [6]:
### Listar tablels em um schema
spark.sql("SHOW TABLES IN iceberg.bronze").show()


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|   bronze|nyc_taxis|      false|
+---------+---------+-----------+



In [7]:
## Inserir valores
query = f"""INSERT INTO iceberg.bronze.nyc_taxis
VALUES (2, 1000371, 1.8, 15.32, 'Y'), (3, 1000372, 2.5, 22.15, 'N'), (5, 1000373, 0.9, 9.01, 'N'), (6, 1000374, 8.4, 42.13, 'Y');"""

display(spark.sql(query))

DataFrame[]

In [8]:
## Consultar dado
spark.sql("SELECT * FROM iceberg.bronze.nyc_taxis where store_and_fwd_flag ='Y' ").show()

+---------+-------+-------------+-----------+------------------+
|vendor_id|trip_id|trip_distance|fare_amount|store_and_fwd_flag|
+---------+-------+-------------+-----------+------------------+
|        6|1000374|          8.4|      42.13|                 Y|
|        2|1000371|          1.8|      15.32|                 Y|
+---------+-------+-------------+-----------+------------------+



In [9]:
partitions = spark.sql("SELECT * FROM iceberg.bronze.nyc_taxis.history").show()


+--------------------+-------------------+---------+-------------------+
|     made_current_at|        snapshot_id|parent_id|is_current_ancestor|
+--------------------+-------------------+---------+-------------------+
|2025-04-13 16:51:...|2179880345541141209|     NULL|               true|
+--------------------+-------------------+---------+-------------------+



In [10]:
spark.stop()